In [1]:
suppressPackageStartupMessages(library(tidyverse))
suppressPackageStartupMessages(library(data.table))


In [2]:
#inputs
phe_info_f <- '../snpnet/pheno.info.tsv'
PheWAS_traits_f <- 'PRS_PheWAS.phenotypes.tsv'
PRS_PheWAS_res_f <- 'data/phewas.white_british.full.tsv'

# output
PRS_PheWAS_summary_f <- 'phewas.white_british.summary.tsv'


In [3]:
# list of target traits
PheWAS_traits <- fread(PheWAS_traits_f, sep='\t') %>%
rename('GBE_ID' = '#GBE_ID')


In [4]:
# read phenotype meta data file
phe_info_df <- fread(phe_info_f) %>%
rename('GBE_ID' = '#GBE_ID')


In [18]:
traits_for_write_up <- c(
    'INI30190',
    'INI30150',
    'INI30210',
    'INI30120',
    'INI30180',
    'INI30140',
    'INI30200',
    'INI30000'
)

In [19]:
phe_info_df %>%
filter(GBE_ID %in% traits_for_write_up)

GBE_ID,pheno,pheno_plot,Units_of_measurement
<chr>,<chr>,<chr>,<chr>
INI30000,White_blood_cell_(leukocyte)_count,White blood cell count,$10^9$ cells/Litre
INI30120,Lymphocyte_count,Lymphocyte count,$10^9$ cells/Litre
INI30140,Neutrophill_count,Neutrophill count,$10^9$ cells/Litre
INI30150,Eosinophill_count,Eosinophill count,$10^9$ cells/Litre
INI30180,Lymphocyte_percentage,Lymphocyte %,percent
INI30190,Monocyte_percentage,Monocyte %,percent
INI30200,Neutrophill_percentage,Neutrophill %,percent
INI30210,Eosinophill_percentage,Eosinophill %,percent


In [20]:
PRS_PheWAS_res_df <- fread(
    PRS_PheWAS_res_f,
    colClasses=c('BETA_str'='character', 'P_str'='character')
) %>%
rename('phenotype' = '#phenotype') %>%
mutate(
    BETA_str = sprintf('%.3f (%.3f,%.3f)', BETA, l_err, u_err),
) %>%
select(-l_err, -u_err, -std_err, -BETA) %>%
rename('BETA'='BETA_str') %>%
filter(PRS_col %in% (traits_for_write_up %>% lapply(function(x){paste0(x, '_PRS')})))


In [21]:
PRS_PheWAS_res_df$fdr <- p.adjust(PRS_PheWAS_res_df$P, method = 'BH')


In [22]:
PRS_PheWAS_res_df$fdr %>% min()

[1] 0.007016558

In [23]:
PRS_PheWAS_res_df %>% 
select(phenotype, PRS_col) %>%
unique() %>% nrow()

[1] 352

We have 352 unique combination of PRS phenotypes and the target (Infectious Diseases) phenotypes.


In [24]:
PRS_PheWAS_summary_df <- PRS_PheWAS_res_df %>% 
arrange(P) %>%
select(-P) %>%
rename('P' = 'P_str') %>%
rename('GBE_ID' = 'phenotype')%>%
left_join(
    PheWAS_traits %>% select(GBE_ID, phenotype, N_WB),
    by='GBE_ID'
)%>%
left_join(
    phe_info_df %>%
    mutate(
        PRS_col = paste0(GBE_ID, '_PRS')
    ) %>%
    select(PRS_col, pheno_plot) %>%
    rename('PRS_pheno'='pheno_plot'),
    by='PRS_col'
) %>%
rename('PRS'='PRS_col') %>%
select(GBE_ID, PRS, phenotype, PRS_pheno, BETA, P, fdr)

In [25]:
PRS_PheWAS_summary_df %>%
rename('#GBE_ID' = 'GBE_ID') %>%
fwrite(PRS_PheWAS_summary_f, sep='\t', na = "NA", quote=F)

